In [1]:
import boto3
import sagemaker
import pandas as pd

# Pushing container to ECR

In [ ]:
!aws ecr get-login-password --region eu-central-1 | docker login --username AWS --password-stdin 164226962805.dkr.ecr.eu-central-1.amazonaws.com

In [ ]:
!docker build -t mentoring-ecr-repo .

In [ ]:
!docker tag mentoring-ecr-repo:latest 164226962805.dkr.ecr.eu-central-1.amazonaws.com/mentoring-ecr-repo:latest

In [ ]:
!docker push 164226962805.dkr.ecr.eu-central-1.amazonaws.com/mentoring-ecr-repo:latest

# Setting up the Endpoint

https://docs.aws.amazon.com/sagemaker/latest/dg/serverless-endpoints-create.html

## Creating the model

In [3]:
client = boto3.client("sagemaker", region_name="eu-central-1")

In [6]:


image_uri = "164226962805.dkr.ecr.eu-central-1.amazonaws.com/mentoring-ecr-repo:latest"

model_name = "gradient-boosting-regressor"

response = client.create_model(
    ModelName = model_name,
    ExecutionRoleArn = "arn:aws:iam::164226962805:role/sagemaker-role",
    Containers = [{
        "Image": image_uri,
        "Mode": "SingleModel"
        #"ModelDataUrl": model_url,
    }]
)

## Creating the Endpoint Configuration

In [9]:
response = client.create_endpoint_config(
   EndpointConfigName="cloud-deployment-endpoint-config",
   #KmsKeyId="arn:aws:sagemaker:us-east-1:164226962805:model/gradient-boosting-regressor",
   ProductionVariants=[
        {
            "ModelName": "gradient-boosting-regressor",
            "VariantName": "AllTraffic",
            "ServerlessConfig": {
                "MemorySizeInMB": 1024,
                "MaxConcurrency": 10,
                "ProvisionedConcurrency": 5,
            }
        } 
    ]
)

## Create the actual Endpoint


In [4]:
response = client.create_endpoint(
    EndpointName="cloud-deployment-endpoint",
    EndpointConfigName="cloud-deployment-endpoint-config"
)

# Getting Inference from the Endpoint

In [5]:
payload_df = pd.read_csv("payload.csv")
payload = payload_df.to_json(orient="split")
payload

'{"columns":["instant","dteday","season","yr","mnth","hr","holiday","weekday","workingday","weathersit","temp","atemp","hum","windspeed","casual","registered"],"index":[0,1,2,3,4,5,6,7,8,9,10,11],"data":[[1,"01\\/01\\/2011",1,0,1,0,0,6,0,1,0.24,0.2879,0.81,0.0,3,13],[2,"01\\/01\\/2011",1,0,1,1,0,6,0,1,0.22,0.2727,0.8,0.0,8,32],[3,"01\\/01\\/2011",1,0,1,2,0,6,0,1,0.22,0.2727,0.8,0.0,5,27],[4,"01\\/01\\/2011",1,0,1,3,0,6,0,1,0.24,0.2879,0.75,0.0,3,10],[5,"01\\/01\\/2011",1,0,1,4,0,6,0,1,0.24,0.2879,0.75,0.0,0,1],[6,"01\\/01\\/2011",1,0,1,5,0,6,0,2,0.24,0.2576,0.75,0.0896,0,1],[7,"01\\/01\\/2011",1,0,1,6,0,6,0,1,0.22,0.2727,0.8,0.0,2,0],[8,"01\\/01\\/2011",1,0,1,7,0,6,0,1,0.2,0.2576,0.86,0.0,1,2],[9,"01\\/01\\/2011",1,0,1,8,0,6,0,1,0.24,0.2879,0.75,0.0,1,7],[10,"01\\/01\\/2011",1,0,1,9,0,6,0,1,0.32,0.3485,0.76,0.0,8,6],[11,"01\\/01\\/2011",1,0,1,10,0,6,0,1,0.38,0.3939,0.76,0.2537,12,24],[12,"01\\/01\\/2011",1,0,1,11,0,6,0,1,0.36,0.3333,0.81,0.2836,26,30]]}'

In [6]:
runtime = boto3.client("sagemaker-runtime")

endpoint_name = "cloud-deployment-endpoint"
content_type = "application/json"

response = runtime.invoke_endpoint(
    EndpointName=endpoint_name,
    ContentType=content_type,
    Body=payload
)

response_body = response["Body"].read().decode().split("\n")
payload_df["predicted"] = response_body[0:-1]
payload_df


,instant,dteday,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,predicted
0,1,01/01/2011,1,0,1,0,0,6,0,1,0.24,0.2879,0.81,0.0000,3,13,77.43092402379384
1,2,01/01/2011,1,0,1,1,0,6,0,1,0.22,0.2727,0.80,0.0000,8,32,47.81864372418429
2,3,01/01/2011,1,0,1,2,0,6,0,1,0.22,0.2727,0.80,0.0000,5,27,68.6624707038762
3,4,01/01/2011,1,0,1,3,0,6,0,1,0.24,0.2879,0.75,0.0000,3,10,47.85333719985825
4,5,01/01/2011,1,0,1,4,0,6,0,1,0.24,0.2879,0.75,0.0000,0,1,35.53473742111073
5,6,01/01/2011,1,0,1,5,0,6,0,2,0.24,0.2576,0.75,0.0896,0,1,31.689727308448912
6,7,01/01/2011,1,0,1,6,0,6,0,1,0.22,0.2727,0.80,0.0000,2,0,19.913885339677364
7,8,01/01/2011,1,0,1,7,0,6,0,1,0.20,0.2576,0.86,0.0000,1,2,-48.430108761425764
8,9,01/01/2011,1,0,1,8,0,6,0,1,0.24,0.2879,0.75,0.0000,1,7,25.826745032662416
9,10,01/01/2011,1,0,1,9,0,6,0,1,0.32,0.3485,0.76,0.0000,8,6,98.94064513523273


# Cleaning up

In [3]:
response = client.delete_endpoint(
    EndpointName="cloud-deployment-endpoint",
)